In [1]:

!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 4.7MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [819 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ P

In [0]:
import pandas as pd
import re
import time
import numpy as np
import random
import sys
sys.path.append('/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# Activation of Chrome Options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-impl-side-painting')

# Fonctions de scrappe

In [17]:
#La session pour les auteurs.
site_auteur= webdriver.Chrome("chromedriver",chrome_options=chrome_options) 

def f_auteur(doc,i):
    """
        Une fonction pour un document donné en paramère renvoie les infos des auteurs
    
    """
    auteurs=[]
    try:
        contributeur=doc.find_elements_by_class_name("contributor")   
        c=1     
        for con in contributeur:
            #Au cas ou le lien n'existe pas
            try:
                #Recuperer le lien et se connecter au site de l'auteur
                lien=con.find_element_by_class_name("authority").get_attribute("href")
                site_auteur.get(lien)

                #Recuperer son numero
                num=lien[32:]

                #Recuper les infos de l'auteur
                infos=site_auteur.find_element_by_class_name("authority-idcard")
                info=infos.find_element_by_xpath("//h2[@itemprop]").text

                #Son nom
                nom=re.search(r"([A-Z]*\w*[, .-]*)*",info).group(0).replace(",","")

                #Date naissance s'il existe
                try:
                    birth=re.search(r"\([0-9]{4}",info).group(0)
                    birth=str(birth)[1:]
                except:
                    birth=''

                #Date de decès s'il existe
                try:
                    death=re.search(r"\-[0-9]{4}",info).group(0)
                    death=str(death)[1:]
                except:
                    death=''

                #Son biographie s'il existe. Cela nous permettra après de recuperer sa profession et son pays d'origine
                try:
                    bio=site_auteur.find_element_by_class_name("authority-bio").text
                except:
                    bio=''
            except:
                num='000'+str(i)+str(c)
                nom=con.find_element_by_class_name("authority").text.replace(",","")
                birth=''
                death=''
                bio=''
                pass
                 
            #Son role s'il exite
            try:
                role=con.find_element_by_class_name("role").text
                role=str(role)[1:-1]
            except NoSuchElementException:
                role=''
                pass
            
            
            auteurs.append([num,nom,birth,death,bio,role])
            c+=1
    except NoSuchElementException:
        auteurs=[['','','','','','']]
        
    return auteurs

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [0]:
def f_article(doc):
    """
        Pour un document donné, la fonction renvoie son titre, date publication ...
    """
    
    #Title
    try:
        title=doc.find_elements_by_class_name("title")[0].text
    except NoSuchElementException:
        title=''
        pass
    
    #Type
    try:
        typee=re.search(r"\[\w+([ ',-]\w+)*\]",doc.find_elements_by_css_selector("div")[0].text).group(0)
        typee=str(typee)[1:-1]
    except:
        typee=''
        pass
    
    #Date de publication
    try:
        datePub=int(doc.find_element_by_class_name("documentYear").text[5:])
    except NoSuchElementException:
        datePub=''
        pass
    
    
    #Numero publication
    try:
        numPub=doc.find_element_by_class_name("documentIssue").text
    except NoSuchElementException:
        numPub=''
        pass

    #Abstract
    try:
        abstract=doc.find_element_by_class_name("searchContext").text
    except NoSuchElementException:
        abstract=''
        pass
    
    return [title,typee,datePub,numPub,abstract]

In [5]:
#Session pour les collections de persee et les collections erudits
site_persee= webdriver.Chrome("chromedriver",chrome_options=chrome_options) 
site_erudit= webdriver.Chrome("chromedriver",chrome_options=chrome_options) 

def f_collection(doc):
    """
        Pour un document donné en paramètre, il renvoie son eissn, nom ...
            
    """
    try:
        #Recuperer le lien et se connecter au site
        clt=doc.find_element_by_class_name("collection")
        lien=clt.find_element_by_css_selector("a").get_attribute("href")
        site_persee.get(lien)

        #Si le site est un site de persée
        try:
            info_box=site_persee.find_element_by_class_name("info-box")
            divs=info_box.find_elements_by_css_selector("div")
            
            annee_deb=int(re.search(r"[0-9]+",divs[0].text).group(0))
            annee_fin=int(re.search(r" [0-9]+",divs[0].text).group(0))
            nb_num=int(re.search(r"[0-9]+",divs[1].text).group(0))
            nb_doc=int(re.search(r"[0-9]+",divs[2].text).group(0))
            try:
                issn1=re.search(r"[0-9]+\-[0-9]+",divs[3].text).group(0)
                ss=0
            except:
                issn1=str(annee_deb)+'-'+str(annee_fin)
                ss=1

            #Cliquer sur INFORMATION de la page persée
            bouton = site_persee.find_element_by_id("tab-header-item-information")
            if bouton : 
                if bouton.is_displayed() :
                    bouton.click()
                    pause()

            table=site_persee.find_element_by_css_selector("table")
            trs=table.find_elements_by_css_selector("tr")
            discipline=trs[2].find_elements_by_css_selector("td")[1].text
            editeur=trs[3].find_elements_by_css_selector("td")[1].text

            try:
                issn2=trs[4].find_elements_by_css_selector("td")[1].text
            except:
                issn2=issn1
                ss=2

            if ss==0:
                issn=issn1
            else:
                issn=iss2
        

            nom_clt=site_persee.find_element_by_id("collection-header").find_element_by_css_selector("h2").text
            collection=[issn,nom_clt,nb_num,nb_doc,annee_deb,annee_fin,discipline,editeur]
        except:
            #Si le site est un site d'érudit
            nom_clt=site_persee.find_element_by_class_name("journal-title").text
        
            #Recuperer le lien de redirection et s'y connecter
            link=site_persee.find_element_by_css_selector("link").get_attribute("href")
            site_erudit.get(link+"#back-issues")
        
        
            back=site_erudit.find_element_by_id("back-issues")
            nb_num=re.search(r"[0-9]+",back.find_element_by_css_selector('h2').text).group(0)
            annees=back.find_elements_by_class_name("year-list__title")
            annee_fin=annees[0].text
            annee_deb=annees[-1].text
            nb_doc=len(back.find_elements_by_css_selector("li"))
        
            fiche=site_erudit.find_elements_by_class_name("sidebar-block")[2].text

            try:
                issn=re.search(r"[0-9]+\-[0-9]+ \(numérique",fiche).group(0)[:-11]
            except:
                issn=str(annee_deb)+'-'+str(annee_fin)

            try:
                editeur=re.search(r"Éditeur[s]?\n.+",fiche).group(0)
                editeur=editeur.split()[1:]
                editeur=' '.join(editeur)    
            except:
                editeur=''        

            try:
                discipline=re.search(r"Discipline[s]?\n.+",fiche).group(0)
                discipline=discipline.split()[1:]
                discipline=' '.join(discipline)
            except:
                discipline=''

            collection=[issn,nom_clt,nb_num,nb_doc,annee_deb,annee_fin,discipline,editeur]
            
    except NoSuchElementException:
        collection=['','','','','','','','']
        pass
    return collection

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [0]:
#Creer les dataframes
df_article = pd.DataFrame(columns=['num_doc','title','type','annee_publication','num_publication','abstract','issn'])
df_auteur= pd.DataFrame(columns=['num','nom','annee_naissance','annee_decès','bio'])
df_redige=pd.DataFrame(columns=['nom','num_doc','role'])
df_collection=pd.DataFrame(columns=['issn','nom_clt','nb_numeros','nb_document','pre_année_pub','der_année_pub','discpline','editeur'])

In [7]:
#mettre le bon chemin 
driver = webdriver.Chrome("chromedriver",chrome_options=chrome_options) 

#recupérer la page web
driver.get("https://www.persee.fr/search?ta=article&q=fuite+des+cerveaux+europe")

# fonction pause
def pause():
    time_break = random.randint(1, 2)
    return time.sleep(time_break)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [0]:
#Initialiser
j=0
M=0
N=0

In [26]:
# CHACUN ESSAYER DE RECUPERER LE MAX QUI LUI EST POSSIBLE.
# CONSEIL: RECUPERER PAR 1000
while N<5000:
    
    # scroller jusqu'en bas de page
    driver.execute_script("window.scrollTo (0, document.body.scrollHeight);")
    docs=driver.find_elements_by_class_name("doc-result")
    N=len(docs)

    #Se positionner au dernière élement recuperé
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center', inline: 'nearest'});", docs[N-1])
    pause()
    if N%500==0:
        print("Recuperer:",N)

Recuperer: 500
Recuperer: 1000
Recuperer: 1500
Recuperer: 2000
Recuperer: 2500
Recuperer: 3000
Recuperer: 3500
Recuperer: 4000
Recuperer: 4500
Recuperer: 5000


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_collection=pd.read_csv('/content/drive/My Drive/Colab Notebooks/collection_v6.csv')
df_article=pd.read_csv('/content/drive/My Drive/Colab Notebooks/documents_v6.csv')
df_auteur=pd.read_csv('/content/drive/My Drive/Colab Notebooks/personne_v6.csv')
df_redige=pd.read_csv('/content/drive/My Drive/Colab Notebooks/redige_v6.csv')

In [0]:
#Reprendre là ou je m'etais arreté
j=len(df_redige)
M=len(df_article)
j,M

In [0]:
#Une fois recuperer, inserer
for i in range(M,N): 
    doc=docs[i]
        
    #Inserer dans le Dataframe collection
    collection=f_collection(doc)
    if collection[0] not in list(df_collection['issn']):
        df_collection.loc[i,:]=collection
    
    #Inserer dans le Dataframe article
    article=f_article(doc)
    article.insert(0,i+1)
    article.append(collection[0])
    df_article.loc[i,:]=article
    
    #Inserer dans le Dataframe auteur
    auteurs=f_auteur(doc,i)
    for auteur in auteurs:
        if auteur[0] not in list(df_auteur['num']):
            df_auteur.loc[j,:]=auteur[:-1]
    #Inserer dans le Dataframe Redige
        df_redige.loc[j,:]=[auteur[0],i+1,auteur[-1]]
        j+=1
  
    if (i+1)%500==0:
       print("Inserer:",i+1)
M=N       

In [0]:
#Tranformer en csv
df_collection.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/collection_v6.csv',index=False)
df_article.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/documents_v6.csv',index=False)
df_auteur.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/personne_v6.csv',index=False)
df_redige.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/redige_v6.csv',index=False)

In [0]:
#Fermer les sessions
#driver.close()
#site_erudit.close()
#site_persee.close()
#site_personne.close()